# Доля аудитории по баинговым группам в 100+
Пример расчета Доли телеканалов по баинговым аудиториям.

## Описание задачи и условий расчета
- Период: 01.08.2022 - 31.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 18+ 100K+, Все 14-44 100K+, Все 11-34 100K+, Все 14-59 100K+, Женщины 14-39 100K+, Женщины 25-59 100K+, Все 25-59 100K+, Мужчины  25-59 100K+, Все 10-45 100K+, Все 25+ 100K+, Все 25-49 100K+, Все 29+ 100K+, Все 4-45 100K+, Все 25-54 100K+, Мужчины  25+ 100K+, Все 25-55 100K+, Все 20-44 100K+, Мужчины  18-45 100K+, Женщины 20-45 100K+, Все 18-44 100K+, Мужчины  18+ 100K+, Женщины 14-44 100K+, Мужчины  14-59 100K+, Все 4-40 100K+, Мужчины  25-49 100K+, Женщины 25-49 100K+, Женщины 18-45 100K+, Все 18-59 100K+
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ, РОССИЯ 1, НТВ, ПЯТЫЙ КАНАЛ, ТНТ 4									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

In [ ]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ПЕРВЫЙ КАНАЛ", "РОССИЯ 1", "НТВ", "ПЯТЫЙ КАНАЛ", "ТНТ 4"])

# далее в запросах будем использовать id телесетей.

In [ ]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол','возраст','100+'])

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (1, 2, 4, 259, 329)'

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [ ]:
# Задаем необходимые группы
targets = {
    'Все 18+ 100K+':'age >= 18 AND cube100Plus100Minus = 1',
    'Все 14-44 100K+':'age >= 14 AND age <= 44 AND cube100Plus100Minus = 1',
    'Все 11-34 100K+':'age >= 11 AND age <= 34 AND cube100Plus100Minus = 1',
    'Все 14-59 100K+':'age >= 14 AND age <= 59 AND cube100Plus100Minus = 1',
    'Женщины  14-39 100K+':'age >= 14 AND age <= 39 AND sex = 2 AND cube100Plus100Minus = 1',
    'Женщины  25-59 100K+':'age >= 25 AND age <= 59 AND sex = 2 AND cube100Plus100Minus = 1',
    'Все 25-59 100K+':'age >= 25 AND age <= 59 AND cube100Plus100Minus = 1',
    'Мужчины 25-59 100K+':'age >= 25 AND age <= 59 AND sex = 1 AND cube100Plus100Minus = 1',
    'Все 10-45 100K+':'age >= 10 AND age <= 45 AND cube100Plus100Minus = 1',
    'Все 25+ 100K+':'age >= 25 AND cube100Plus100Minus = 1',
    'Все 25-49 100K+':'age >= 25 AND age <= 49 AND cube100Plus100Minus = 1',
    'Все 29+ 100K+':'age >= 29 AND cube100Plus100Minus = 1',
    'Все 4-45 100K+':'age >= 4 AND age <= 45 AND cube100Plus100Minus = 1',
    'Все 25-54 100K+':'age >= 25 AND age <= 54 AND cube100Plus100Minus = 1',
    'Мужчины 25+ 100K+':'age >= 25 AND sex = 1 AND cube100Plus100Minus = 1',
    'Все 25-55 100K+':'age >= 25 AND age <= 55 AND cube100Plus100Minus = 1',
    'Все 20-44 100K+':'age >= 20 AND age <= 44 AND cube100Plus100Minus = 1',
    'Мужчины 18-45 100K+':'age >= 18 AND age <= 45 AND sex = 1 AND cube100Plus100Minus = 1',
    'Женщины  20-45 100K+':'age >= 20 AND age <= 45 AND sex = 2 AND cube100Plus100Minus = 1',
    'Все 18-44 100K+':'age >= 18 AND age <= 44 AND cube100Plus100Minus = 1',
    'Мужчины 18+ 100K+':'age >= 18 AND sex = 1 AND cube100Plus100Minus = 1',
    'Женщины  14-44 100K+':'age >= 14 AND age <= 44 AND sex = 2 AND cube100Plus100Minus = 1',
    'Мужчины 14-59 100K+':'age >= 14 AND age <= 59 AND sex = 1 AND cube100Plus100Minus = 1',
    'Все 4-40 100K+':'age >= 4 AND age <= 40 AND cube100Plus100Minus = 1',
    'Мужчины 25-49 100K+':'age >= 25 AND age <= 49 AND sex = 1 AND cube100Plus100Minus = 1',
    'Женщины  25-49 100K+':'age >= 25 AND age <= 49 AND sex = 2 AND cube100Plus100Minus = 1',
    'Женщины  18-45 100K+':'age >= 18 AND age <= 45 AND sex = 2 AND cube100Plus100Minus = 1',
    'Все 18-59 100K+':'age >= 18 AND age <= 59 AND cube100Plus100Minus = 1'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{target}" 
    basedemo_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Настраиваем внешний вид таблицы

In [ ]:
df_res = pd.pivot_table(df, values=['Share'],
                        index=['prj_name'], 
                        columns=['tvNetName'])
df_res

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_share_buying_targets')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)